In [55]:
import numpy as np
from numpy import linalg
from sympy import *

In [56]:
# Rossenbrock function
# f1(x) = 10*(x2 - x1^2)
# f2(x) = 1 - x1
# x0 = (-1.2, 1)
# f = 0 at (1, 1)
# n = 2, m = 2 13

In [57]:
# Freudenstein and Roth function
# f1(x) = -13 + x1 + ((5 - x2)*x2 - 2)*x2
# f2(x) = -29 + x1 + ((x2 + 1)*x2 - 14)*x2
# x0 = (0.5, -2)
# f = 0 at (5, 4)

In [58]:
# Powell badly scaled function
# f1(x) = 10**4*x1*x2 - 1
# f2(x) = exp(-x1) + exp(-x2) - 1.0001
# x0 = (0, 1)
# f = 0 at (1.098...10**-5, 9.106...)

In [59]:
# Powell singular function
# f1(x) = x1 + 10*x2
# f2(x) = 5^(1/2)*(x3 - x4)
# f3(x) = (x2 - 2*x3)^2
# f4(x) = 10^(1/2)*(x1 - x4)^2
# x0 = [3, -1, 0, 1]
# f = 0

In [60]:
x1 = symbols('x1')
x2 = symbols('x2')
x3 = symbols('x3')
x4 = symbols('x4')

# f1 = simplify("10*(x2 - x1^2)")
# f2 = sympify("1 - x1")
# x0 = np.array([-3, 2], dtype = 'float')

# f1 = simplify("(10**4)*x1*x2 - 1")
# f2 = simplify("exp(-x1) + exp(-x2) - 1.0001")
# x0 = np.array([0, 1], dtype = 'float')

# f1 = simplify("-13 + x1 + ((5 - x2)*x2 - 2)*x2")
# f2 = simplify("-29 + x1 + ((x2 + 1)*x2 - 14)*x2")
# x0 = np.array([0.5, -4], dtype = 'float')

f1 = simplify('x1 + 10*x2')
f2 = simplify('5^(1/2)*(x3 - x4)')
f3 = simplify('(x2 - 2*x3)^2')
f4 = simplify('10^(1/2)*(x1 - x4)^2')
x0 = np.array([3, -1, 0, 1], dtype = 'float')

# x = np.array([x1, x2])
# F = np.array([f1, f2])

x = np.array([x1, x2, x3, x4])
F = np.array([f1, f2, f3, f4])

In [61]:
# n = int(input("Enter n = "))
# x = np.array([symbols('x{0}'.format(i + 1)) for i in range(n)])

# F = np.array([simplify(input("f{0} = ".format(i + 1))) for i in range(n)])
# x0 = np.array([float(input("{0} Element of x0".format(i + 1))) for i in range(n)], dtype = 'float')

eps = 10 ** -7

In [62]:
F_0 = lambda x0: np.array([F[i].subs(dict(zip(x, x0))) for i in range(len(F))], dtype = 'float')

In [63]:
def F_Derivative(x0):
    F_der = np.array([[diff(F[i], x[j]) for j in range(len(x))] for i in range(len(F))])
    return np.array([[F_der[i][j].subs(dict(zip(x, x0))) for j in range(len(F_der[i]))] for i in range(len(F_der))], dtype = 'float')

In [64]:
# test function for F(x, y)
# A = np.array([simplify("x1**2 + 7*x1 + 2*x1*x2 - 3"), simplify("x1 + x2**3")])
# A = np.array([simplify("-13 + x1 + ((5 - x2)*x2 - 2)*x2"), simplify("-29 + x1 + ((x2 + 1)*x2 - 14)*x2")])
# A[0].subs({x1: 1.13975640, x2: 1.05684169})

In [65]:
# Поділена різниця першого порядку
def DividedDifferenceOfOperator(x1, x2):
    assert(len(x1) == len(x2))
    arr = []
    for i in range(len(F)):
        subarr = []
        for j in range(len(x1)):
            params1 = [*x1[0:j+1], *x2[j+1:len(x2)+1]]
            params2 = [*x1[0:j], *x2[j:len(x2)+1]]
            # print("x1[j] = {0}, x2[j] = {1}, x1[j] - x2[j] = {2}".format(x1[j], x2[j], x1[j] - x2[j]))
            # print((F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))))
            result = (F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))) / (x1[j] - x2[j])
            subarr.append(result)
        
        arr.append(subarr)
    
    return np.array(arr, dtype = 'float')


In [66]:
def NewtonMethod(x0, x, F):
    I = 0

    x1 = x0 - np.dot(np.linalg.inv(F_Derivative(x0)), F_0(x0))
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I +=1
    
    while True:
        x0 = x1
        x1 = x0 - np.dot(np.linalg.inv(F_Derivative(x0)), F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [67]:
def PotraMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    I = 0

    matrix = (DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1)
    x1 = x0 - np.dot(np.linalg.inv(matrix), F_0(x0))
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I += 1

    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1

        matrix = (DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1)
        x1 = x0 - np.dot(np.linalg.inv(matrix), F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0


In [68]:
# Inverse operator methods
def SequentialNewtonInverseOperatorMethod(x0, x, F):
    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv(F_Derivative(x0))

    x1 = x0 - np.dot(A_0, F_0(x0))
    right_matrix = 2 * I - np.dot(F_Derivative(x1), A_0)
    A_1 = np.dot(A_0, right_matrix)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))

    number_of_iterations += 1
    
    while True:
        x0 = x1
        A_0 = A_1

        x1 = x0 - np.dot(A_0, F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        right_matrix = 2 * I - np.dot(F_Derivative(x1), A_0)
        A_1 = np.dot(A_0, right_matrix)
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
        number_of_iterations += 1

    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0
    

In [69]:
def ParallelNewtonInverseOperatorMethod(x0, x, F):
    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv(F_Derivative(x0))

    x1 = x0 - np.dot(A_0, F_0(x0))
    right_matrix = 2 * I - np.dot(F_Derivative(x0), A_0)
    A_1 = np.dot(A_0, right_matrix)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))

    number_of_iterations += 1
    
    while True:
        x0 = x1
        A_0 = A_1

        x1 = x0 - np.dot(A_0, F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        right_matrix = 2 * I - np.dot(F_Derivative(x0), A_0)
        A_1 = np.dot(A_0, right_matrix)
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
        number_of_iterations += 1

    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0

In [70]:
def SequentialPotraInverseOperatorMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv((DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1))

    x1 = x0 - np.dot(A_0, F_0(x0))
    matrix = (DividedDifferenceOfOperator(x1, x0) + DividedDifferenceOfOperator(x_1, x1)) - DividedDifferenceOfOperator(x_1, x0)
    
    
    right_matrix = 2 * I - np.dot(matrix, A_0)
    A_1 = np.dot(A_0, right_matrix)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
    number_of_iterations +=1
    
    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1
        A_0 = A_1

        x1 = x0 - np.dot(A_0, F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        matrix = (DividedDifferenceOfOperator(x1, x0) + DividedDifferenceOfOperator(x_1, x1)) - DividedDifferenceOfOperator(x_1, x0)
        # right_matrix = 2 * I - np.dot(F_Derivative(x1), A_0)
        right_matrix = 2 * I - np.dot(matrix, A_0)
        A_1 = np.dot(A_0, right_matrix)
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
        number_of_iterations += 1
    
    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0


In [71]:
def ParallelPotraInverseOperatorMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv((DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1))

    x1 = x0 - np.dot(A_0, F_0(x0))
    # matrix = (F_1(x1, x0) + F_1(x_1, x1)) - F_1(x_1, x0)
    matrix = (DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1)
    # right_matrix = 2 * I - np.dot(F_Derivative(x1), A_0)
    right_matrix = 2 * I - np.dot(matrix, A_0)
    A_1 = np.dot(A_0, right_matrix)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
    number_of_iterations +=1
    
    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1
        A_0 = A_1

        x1 = x0 - np.dot(A_0, F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        # matrix = (F_1(x1, x0) + F_1(x_1, x1)) - F_1(x_1, x0)
        matrix = (DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1)
        # right_matrix = 2 * I - np.dot(F_Derivative(x1), A_0)
        right_matrix = 2 * I - np.dot(matrix, A_0)
        A_1 = np.dot(A_0, right_matrix)
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
        number_of_iterations += 1
    
    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0

In [72]:
def GaussNewtonLeastSquaresMethod(x0, x, F):
    I = 0

    operatorsMultiplication = np.dot(F_Derivative(x0).transpose(), F_Derivative(x0))

    multiplicationOfOperatorAndFunction = np.dot(F_Derivative(x0).transpose(), F_0(x0))

    x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)
    
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I +=1
    
    while True:
        x0 = x1

        operatorsMultiplication = np.dot(F_Derivative(x0).transpose(), F_Derivative(x0))
        multiplicationOfOperatorAndFunction = np.dot(F_Derivative(x0).transpose(), F_0(x0))

        x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)

        if (max(np.abs(x1 - x0)) < eps):
            break

        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [73]:
def PotraLeastSquaresMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)
    
    I = 0

    A  = (DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1)
    operatorsMultiplication = np.dot(A.transpose(), A)
    multiplicationOfOperatorAndFunction = np.dot(A.transpose(), F_0(x0))

    x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I += 1

    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1

        A  = (DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1)
        operatorsMultiplication = np.dot(A.transpose(), A)
        multiplicationOfOperatorAndFunction = np.dot(A.transpose(), F_0(x0))

        x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)

        if (max(np.abs(x1 - x0)) < eps):
            break
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0


In [74]:
print("Метод Гаусса - Ньютона (найменших квадратів)\n\n")
a = GaussNewtonLeastSquaresMethod(x0, x, F)
print(a)

Метод Гаусса - Ньютона (найменших квадратів)


1 Ітерація: Значення F = [ 6.21724894e-15 -1.19161639e-14  2.50000000e-01  3.16227766e+00] у точці x = [ 1.19047619 -0.11904762  0.19047619  0.19047619]
2 Ітерація: Значення F = [1.77635684e-15 1.98602732e-15 6.25000000e-02 7.90569415e-01] у точці x = [ 0.5952381  -0.05952381  0.0952381   0.0952381 ]
3 Ітерація: Значення F = [3.33066907e-16 9.93013661e-16 1.56250000e-02 1.97642354e-01] у точці x = [ 0.29761905 -0.0297619   0.04761905  0.04761905]
4 Ітерація: Значення F = [ 5.55111512e-17 -9.93013661e-16  3.90625000e-03  4.94105884e-02] у точці x = [ 0.14880952 -0.01488095  0.02380952  0.02380952]
5 Ітерація: Значення F = [2.22044605e-16 0.00000000e+00 9.76562500e-04 1.23526471e-02] у точці x = [ 0.07440476 -0.00744048  0.01190476  0.01190476]
6 Ітерація: Значення F = [0.         0.         0.00024414 0.00308816] у точці x = [ 0.03720238 -0.00372024  0.00595238  0.00595238]
7 Ітерація: Значення F = [-3.12250226e-17  0.00000000e+00  6.103515

In [75]:
print("Метод Потра (найменших квадратів)\n\n")
a = PotraLeastSquaresMethod(x0, x, F)
print(a)

Метод Потра (найменших квадратів)


1 Ітерація: Значення F = [-8.14903700e-12  4.96506831e-14  2.50000000e-01  3.16227766e+00] у точці x = [ 1.19047619 -0.11904762  0.19047619  0.19047619]
2 Ітерація: Значення F = [2.37898590e-12 1.98602732e-14 6.25000000e-02 7.90569415e-01] у точці x = [ 0.5952381  -0.05952381  0.0952381   0.0952381 ]
3 Ітерація: Значення F = [1.11022302e-16 9.93013661e-16 1.56250000e-02 1.97642354e-01] у точці x = [ 0.29761905 -0.0297619   0.04761905  0.04761905]
4 Ітерація: Значення F = [8.32667268e-17 0.00000000e+00 3.90625000e-03 4.94105884e-02] у точці x = [ 0.14880952 -0.01488095  0.02380952  0.02380952]
5 Ітерація: Значення F = [ 9.71445147e-17 -2.48253415e-16  9.76562500e-04  1.23526471e-02] у точці x = [ 0.07440476 -0.00744048  0.01190476  0.01190476]
6 Ітерація: Значення F = [-6.24500451e-17 -1.24126708e-16  2.44140625e-04  3.08816178e-03] у точці x = [ 0.03720238 -0.00372024  0.00595238  0.00595238]
7 Ітерація: Значення F = [-2.08166817e-17  0.00000000e+00 

In [76]:
# print("Метод Ньютона з послідовною апроксимацією оберненого оператора\n\n")
# a = SequentialNewtonInverseOperatorMethod(x0, x, F)
# print(a)
# print("Норма відхилу: {0}".format(max(np.abs(a - x0))))

In [77]:
# print("Метод Ньютона з паралельною апроксимацією оберненого оператора\n\n")
# a = ParallelNewtonInverseOperatorMethod(x0, x, F)
# print(a)
# print("Норма відхилу: {0}".format(a - x0))

In [78]:
# print("Метод Потра з послідовною апроксимацією оберненого оператора\n\n")
# a = SequentialPotraInverseOperatorMethod(x0, x, F)
# print(a)
# print("Норма відхилу: {0}".format(max(np.abs(a - x0))))

In [79]:
# print("Метод Потра з паралельною апроксимацією оберненого оператора\n\n")
# a = ParallelPotraInverseOperatorMethod(x0, x, F)
# print(a)
# print("Норма відхилу: {0}".format(max(np.abs(a - x0))))

In [80]:
# print("Метод Ньютона\n\n")
# print(NewtonMethod(x0, x, F))

In [81]:
# print("Метод Потра\n\n")
# print(PotraMethod(x0, x, F))